## 下载数据集

In [ ]:
!rm -rf Glomeruli-dataset.zip Glomeruli-dataset
!wget https://zihao-openmmlab.obs.cn-east-3.myhuaweicloud.com/20230130-mmseg/dataset/Glomeruli-dataset.zip
!unzip Glomeruli-dataset.zip -d data # 解压
!rm -rf Glomeruli-dataset.zip # 删除压缩包

## 导入工具包

In [ ]:
import os

import cv2
import numpy as np
from PIL import Image
from tqdm import tqdm

import matplotlib.pyplot as plt
%matplotlib inline

import os.path as osp
from tqdm import tqdm

import mmcv
import mmengine
from mmseg.apis import init_model, inference_model, show_result_pyplot

## 指定图像和标注文件夹路径

In [ ]:
PATH_IMAGE = 'data/Glomeruli-dataset/images'
PATH_MASKS = 'data/Glomeruli-dataset/masks'

In [ ]:
print('图像个数', len(os.listdir(PATH_IMAGE)))
print('标注个数', len(os.listdir(PATH_MASKS)))

## 查看单张图像及其语义分割标注

In [ ]:
# 指定图像文件名
file_name = 'SAS_21883_001_10.png'

In [ ]:
img_path = os.path.join(PATH_IMAGE, file_name)
mask_path = os.path.join(PATH_MASKS, file_name)

print('图像路径', img_path)
print('标注路径', mask_path)

In [ ]:
img = cv2.imread(img_path)
mask = cv2.imread(mask_path)

In [ ]:
img.shape

In [ ]:
# 可视化图像
plt.imshow(img)
plt.show()

In [ ]:
mask.shape

In [ ]:
# mask 语义分割标注，与原图大小相同，0 为 背景， 1 为 肾小球
np.unique(mask)

在本数据集中，只有一部分图像有肾小球语义分割标注（即mask中值为1的像素），其余图像mask的值均为0

In [ ]:
# 可视化语义分割标注
plt.imshow(mask[:,:,0])
plt.show()

In [ ]:
# 可视化语义分割标注
plt.imshow(mask*255)
plt.show()

## 可视化单张图像及其语义分割标注-代码模板

In [ ]:
plt.imshow(img)
plt.imshow(mask*255, alpha=0.5) # alpha 高亮区域透明度，越小越接近原图
plt.title(file_name)
plt.axis('off')
plt.show()

## 可视化模板-有前景标注

In [ ]:
# n行n列可视化
n = 7

# 标注区域透明度
opacity = 0.5

fig, axes = plt.subplots(nrows=n, ncols=n, sharex=True, figsize=(12,12))

i = 0

for file_name in os.listdir(PATH_IMAGE):
    
    # 载入图像和标注
    img_path = os.path.join(PATH_IMAGE, file_name)
    mask_path = os.path.join(PATH_MASKS, file_name)
    img = cv2.imread(img_path)
    mask = cv2.imread(mask_path)
    
    if 1 in mask:
        axes[i//n, i%n].imshow(img)
        axes[i//n, i%n].imshow(mask*255, alpha=opacity)
        axes[i//n, i%n].axis('off') # 关闭坐标轴显示
        i += 1
    if i > n**2-1:
        break
fig.suptitle('Image and Semantic Label', fontsize=30)
plt.tight_layout()
plt.show()

## 可视化模板-无论前景是否有标注

In [ ]:
# n行n列可视化
n = 10

# 标注区域透明度
opacity = 0.5

fig, axes = plt.subplots(nrows=n, ncols=n, sharex=True, figsize=(12,12))

for i, file_name in enumerate(os.listdir(PATH_IMAGE)[:n**2]):
    
    # 载入图像和标注
    img_path = os.path.join(PATH_IMAGE, file_name)
    mask_path = os.path.join(PATH_MASKS, file_name)
    img = cv2.imread(img_path)
    mask = cv2.imread(mask_path)
    
    # 可视化
    axes[i//n, i%n].imshow(img)
    axes[i//n, i%n].imshow(mask*255, alpha=opacity)
    axes[i//n, i%n].axis('off') # 关闭坐标轴显示
fig.suptitle('Image and Semantic Label', fontsize=30)
plt.tight_layout()
plt.show()

## 获取全部数据文件名列表

In [ ]:
import random
PATH_IMAGE = 'data/Glomeruli-dataset/images'

In [ ]:
all_file_list = os.listdir(PATH_IMAGE)
all_file_num = len(all_file_list)
random.shuffle(all_file_list) # 随机打乱全部数据文件名列表

## 指定训练集和测试集比例

In [ ]:
train_ratio = 0.8
test_ratio = 1 - train_ratio

In [ ]:
train_file_list = all_file_list[:int(all_file_num*train_ratio)]
test_file_list = all_file_list[int(all_file_num*train_ratio):]

In [ ]:
print('数据集图像总数', all_file_num)
print('训练集划分比例', train_ratio)
print('训练集图像个数', len(train_file_list))
print('测试集图像个数', len(test_file_list))

In [ ]:
train_file_list[:5]

In [ ]:
test_file_list[:5]

## 生成两个txt划分文件

In [ ]:
os.mkdir('data/Glomeruli-dataset/splits')

In [ ]:
with open('data/Glomeruli-dataset/splits/train.txt', 'w') as f:
    f.writelines(line.split('.')[0] + '\n' for line in train_file_list)
with open('data/Glomeruli-dataset/splits/val.txt', 'w') as f:
    f.writelines(line.split('.')[0] + '\n' for line in test_file_list)

## MMSegmentation训练语义分割模型

In [ ]:
# 数据集图片和标注路径
data_root = 'data/Glomeruli-dataset'
img_dir = 'images'
ann_dir = 'masks'

# 类别和对应的颜色
classes = ('background', 'glomeruili')
palette = [[128, 128, 128], [151, 189, 8]]

## 修改config配置文件

In [ ]:
# 下载 config 文件 和 预训练模型checkpoint权重文件
!mim download mmsegmentation --config pspnet_r50-d8_4xb2-40k_cityscapes-512x1024 --dest configs/pspnet/

In [ ]:
from mmengine import Config
cfg = Config.fromfile('configs/pspnet/pspnet_r50-d8_4xb2-40k_cityscapes-512x1024.py')

In [ ]:
cfg.norm_cfg = dict(type='BN', requires_grad=True) # 只使用GPU时，BN取代SyncBN
cfg.crop_size = (256, 256)
cfg.model.data_preprocessor.size = cfg.crop_size
cfg.model.backbone.norm_cfg = cfg.norm_cfg
cfg.model.decode_head.norm_cfg = cfg.norm_cfg
cfg.model.auxiliary_head.norm_cfg = cfg.norm_cfg
# modify num classes of the model in decode/auxiliary head
cfg.model.decode_head.num_classes = 2
cfg.model.auxiliary_head.num_classes = 2

# 修改数据集的 type 和 root
cfg.dataset_type = 'StanfordBackgroundDataset'
cfg.data_root = data_root

cfg.train_dataloader.batch_size = 8

cfg.train_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='LoadAnnotations'),
    dict(type='RandomResize', scale=(320, 240), ratio_range=(0.5, 2.0), keep_ratio=True),
    dict(type='RandomCrop', crop_size=cfg.crop_size, cat_max_ratio=0.75),
    dict(type='RandomFlip', prob=0.5),
    dict(type='PackSegInputs')
]

cfg.test_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='Resize', scale=(320, 240), keep_ratio=True),
    # add loading annotation after ``Resize`` because ground truth
    # does not need to do resize data transform
    dict(type='LoadAnnotations'),
    dict(type='PackSegInputs')
]


cfg.train_dataloader.dataset.type = cfg.dataset_type
cfg.train_dataloader.dataset.data_root = cfg.data_root
cfg.train_dataloader.dataset.data_prefix = dict(img_path=img_dir, seg_map_path=ann_dir)
cfg.train_dataloader.dataset.pipeline = cfg.train_pipeline
cfg.train_dataloader.dataset.ann_file = 'splits/train.txt'

cfg.val_dataloader.dataset.type = cfg.dataset_type
cfg.val_dataloader.dataset.data_root = cfg.data_root
cfg.val_dataloader.dataset.data_prefix = dict(img_path=img_dir, seg_map_path=ann_dir)
cfg.val_dataloader.dataset.pipeline = cfg.test_pipeline
cfg.val_dataloader.dataset.ann_file = 'splits/val.txt'

cfg.test_dataloader = cfg.val_dataloader


# 载入预训练模型权重
cfg.load_from = 'configs/pspnet/pspnet_r50-d8_512x1024_40k_cityscapes_20200605_003338-2966598c.pth'

# 工作目录
cfg.work_dir = './work_dirs/tutorial'

# 训练迭代次数
cfg.train_cfg.max_iters = 800
# 评估模型间隔
cfg.train_cfg.val_interval = 400
# 日志记录间隔
cfg.default_hooks.logger.interval = 100
# 模型权重保存间隔
cfg.default_hooks.checkpoint.interval = 400

# 随机数种子
cfg['randomness'] = dict(seed=0)

## 查看完整config配置文件

In [ ]:
print(cfg.pretty_text)

## 保存config配置文件

In [ ]:
cfg.dump('configs/pspnet/mouse_new_cfg.py')

## 读取config

In [ ]:
from mmengine import Config
cfg = Config.fromfile('configs/pspnet/mouse_new_cfg.py')

## 准备训练

In [ ]:
from mmengine.runner import Runner
from mmseg.utils import register_all_modules

# register all modules in mmseg into the registries
# do not init the default scope here because it will be init in the runner
register_all_modules(init_default_scope=False)
runner = Runner.from_cfg(cfg)

## 开始训练

如果遇到报错`CUDA out of memeory`，重启实例或使用显存更高的实例即可。

In [ ]:
runner.train()

## 载入模型

In [ ]:
# 载入 config 配置文件
from mmengine import Config
cfg = Config.fromfile('configs/pspnet/mouse_new_cfg.py')

In [ ]:
from mmengine.runner import Runner
from mmseg.utils import register_all_modules

# register all modules in mmseg into the registries
# do not init the default scope here because it will be init in the runner

register_all_modules(init_default_scope=False)
runner = Runner.from_cfg(cfg)

In [ ]:
# 初始化模型
checkpoint_path = './work_dirs/tutorial/iter_800.pth'
model = init_model(cfg, checkpoint_path, 'cuda:0')

## 载入测试集图像，或新图像

In [ ]:
img = mmcv.imread('data/Glomeruli-dataset/images/VUHSK_1702_39.png')

## 语义分割预测

In [ ]:
result = inference_model(model, img)

In [ ]:
result.keys()

In [ ]:
pred_mask = result.pred_sem_seg.data[0].cpu().numpy()

In [ ]:
pred_mask.shape

In [ ]:
np.unique(pred_mask)

## 可视化语义分割预测结果

In [ ]:
plt.imshow(pred_mask)
plt.show()

In [ ]:
# 可视化预测结果
visualization = show_result_pyplot(model, img, result, opacity=0.7, out_file='pred.jpg')
plt.imshow(mmcv.bgr2rgb(visualization))
plt.show()

## 语义分割预测结果-连通域分析

In [ ]:
plt.imshow(np.uint8(pred_mask))
plt.show()

In [ ]:
connected = cv2.connectedComponentsWithStats(np.uint8(pred_mask), connectivity=4)

In [ ]:
# 连通域个数（第一个有可能是全图，可以忽略）
connected[0]

In [ ]:
# 用整数表示每个连通域区域
connected[1].shape

In [ ]:
np.unique(connected[1])

In [ ]:
plt.imshow(connected[1])
plt.show()

In [ ]:
# 每个连通域外接矩形的左上角X、左上角Y、宽度、高度、面积
connected[2]

In [ ]:
# 每个连通域的质心坐标
connected[3]

## 获取测试集标注

In [ ]:
label = mmcv.imread('data/Glomeruli-dataset/masks/VUHSK_1702_39.png')

In [ ]:
label_mask = label[:,:,0]

In [ ]:
label_mask.shape

In [ ]:
np.unique(label_mask)

In [ ]:
plt.imshow(label_mask)
plt.show()

## 对比测试集标注和语义分割预测结果

In [ ]:
# 测试集标注
label_mask.shape

In [ ]:
# 语义分割预测结果
pred_mask.shape

In [ ]:
# 真实为前景，预测为前景
TP = (label_mask == 1) & (pred_mask==1)

In [ ]:
# 真实为背景，预测为背景
TN = (label_mask == 0) & (pred_mask==0)

In [ ]:
# 真实为前景，预测为背景
FN = (label_mask == 1) & (pred_mask==0)

In [ ]:
# 真实为背景，预测为前景
FP = (label_mask == 0) & (pred_mask==1)

In [ ]:
plt.imshow(TP)
plt.show()

In [ ]:
confusion_map = TP * 255 + FP * 150 + FN * 80 + TN * 10

In [ ]:
plt.imshow(confusion_map)
plt.show()

## 混淆矩阵

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
confusion_matrix_model = confusion_matrix(label_mask.flatten(), pred_mask.flatten())

In [ ]:
import itertools
def cnf_matrix_plotter(cm, classes, cmap=plt.cm.Blues):
    """
    传入混淆矩阵和标签名称列表，绘制混淆矩阵
    """
    plt.figure(figsize=(10, 10))
    
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    # plt.colorbar() # 色条
    tick_marks = np.arange(len(classes))
    
    plt.title('Confusion Matrix', fontsize=30)
    plt.xlabel('Pred', fontsize=25, c='r')
    plt.ylabel('True', fontsize=25, c='r')
    plt.tick_params(labelsize=16) # 设置类别文字大小
    plt.xticks(tick_marks, classes, rotation=90) # 横轴文字旋转
    plt.yticks(tick_marks, classes)
    
    # 写数字
    threshold = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > threshold else "black",
                 fontsize=12)

    plt.tight_layout()

    plt.savefig('混淆矩阵.pdf', dpi=300) # 保存图像
    plt.show()

In [ ]:
classes = ('background', 'glomeruili')

In [ ]:
cnf_matrix_plotter(confusion_matrix_model, classes, cmap='Blues')

## 添加数据集类

In [ ]:
# 数据集配置文件
!wget https://zihao-openmmlab.obs.cn-east-3.myhuaweicloud.com/20230130-mmseg/dataset/StanfordBackgroundDataset.py -O mmseg/datasets/StanfordBackgroundDataset.py

In [ ]:
# 修改 ../mmsegmentation/mmseg/datasets/__init__.py，添加数据集
!wget https://zihao-openmmlab.obs.cn-east-3.myhuaweicloud.com/20230130-mmseg/dataset/__init__.py -O mmseg/datasets/__init__.py

## 测试集精度指标

In [ ]:
!python tools/test.py configs/pspnet/mouse_new_cfg.py ./work_dirs/tutorial/iter_800.pth

## 速度指标-FPS

In [ ]:
!python tools/analysis_tools/benchmark.py configs/pspnet/mouse_new_cfg.py ./work_dirs/tutorial/iter_800.pth